In [1]:
import math
import numpy as np
import pathlib
import scipy.io.wavfile as wav

from python_speech_features import mfcc
from numpy.random import random
from scipy.stats import multivariate_normal
from tqdm.notebook import tqdm

In [2]:
PATH = pathlib.Path('~/Downloads/syllable/SVL-PRASANNA-CV').expanduser() # Path to dataset
CLASSES = ('a', 'xho') # Two classes to distinguish
X_Dim = 13 # Basic 13 element version of MFCC
DELTA = 0.0000000001 # Delta used for numerical stability

In [3]:
def normpdf(x, mean, sd):
    var = sd**2 + DELTA
    return -0.5*(np.log(2*np.pi*var) + (x-mean)**2/var)

# Part 1

Find Mel-Frequency Cepstral Coefficients (MFCCs) from the raw speech samples.  
Pick 25 ms worth of speech samples with a 10 ms overlap to find MFCCs.  
Use the basic 13 element version of MFCC as the feature vector representing 25 ms of speech.

In [4]:
def read_data():
    """
    Read wav files and create a dataset
    """
    dataset = []
    for name in CLASSES:
        data = []
        for filepath in (PATH / name).glob('*.wav'):
            rate, sig = wav.read(filepath)
            mfcc_features = mfcc(sig, rate, winlen=0.025, winstep=0.015, numcep=X_Dim)
            data.append(mfcc_features)
        dataset.append(data)
    return dataset


In [5]:
class HMM:

    def __init__(self, N=100):
        self.N = N
        self.pi = np.asarray([1./self.N] * self.N)
        self.a = np.random.rand(self.N, self.N)
        self.a = self.a/np.sum(self.a, axis=1)
        self.mu = np.zeros(self.N)
        self.covat = np.ones(self.N)

    def update(self, sequence):
        # Calculate Update
        forward = self.forward(sequence)
        backward = self.backward(sequence)
        gamma, ksi = self.update_gamma_ksi(forward,backward,sequence)
        for s in range(self.N):
            self.pi[s] = gamma[s, 0]
            for s_ in range(self.N):
                self.a[s,s_] = self.log_sum_exp(ksi[s,s_,:]) - self.log_sum_exp(gamma[s,:])

        for s in range(self.N):
            gamma_s = gamma[s,:]
            self.mu[s] = np.sum(np.exp(gamma_s)*sequence)/(np.sum(np.exp(gamma_s)) + DELTA)
            self.covat[s] = np.sum(np.exp(gamma_s)*(sequence-self.mu[s])*(sequence-self.mu[s]))/(np.sum(np.exp(gamma_s)) + DELTA)


    def forward(self, sequence):
        """
        Forward Algorithm to calculate the alpha matrix of size N * X_Dim
        """
        forward = np.zeros((self.N, X_Dim))
        for s in range(self.N):
            forward[s,0] = self.pi[s] + normpdf(sequence[0],self.mu[s],self.covat[s])

        for t in range(1, X_Dim):
            o = sequence[t]
            for s in range(self.N):
                sum_seq = []
                for s_ in range(self.N):
                    sum_seq.append(forward[s_,t-1] + self.a[s_][s])
                    
            forward[s,t] = self.log_sum_exp(sum_seq) + normpdf(o,self.mu[s],self.covat[s])
        return forward

    def backward(self, sequence):
        """
        Backward Algorithm method to calculate the bksi matrix of shape N * X_Dim
        """
        backward = np.zeros((self.N, X_Dim))
        for t in range(X_Dim-2,0,-1):
            o = sequence[t+1]
            for s in range(self.N):
                sum_seq = []
                for s_ in range(self.N):
                    sum_seq.append(backward[s_,t+1] + self.a[s,s_] + normpdf(o,self.mu[s_],self.covat[s_]))
            
            backward[s,t] = self.log_sum_exp(sum_seq)
        return backward

    def update_gamma_ksi(self, forward, backward, sequence):
        gamma = np.zeros((self.N, X_Dim))
        ksi = np.zeros((self.N,self.N, X_Dim))

        for t in range(X_Dim):
            sum_ = self.log_sum_exp(forward[:,t]+ backward[:,t])
            for s in range(self.N):
                gamma[s,t] = forward[s,t] + backward[s,t] -sum_

        sum_ = np.zeros(X_Dim-1)
        for t in range(X_Dim-1):
            sum_seq = []
            for s in range(self.N):
                for s_ in range(self.N):
                   sum_seq.append(forward[s,t] + self.a[s,s_] + backward[s_,t+1] + normpdf(sequence[t+1],self.mu[s_],self.covat[s_])) 
            sum_[t] = self.log_sum_exp(sum_seq)

        for t in range(X_Dim-1):
            for s in range(self.N):
                for s_ in range(self.N):
                    ksi[s,s_,t] = forward[s,t] + self.a[s,s_] + backward[s_,t+1] +  normpdf(sequence[t+1],self.mu[s_],self.covat[s_]) - sum_[t]

        return gamma,ksi

    def log_sum_exp(self, seq):
        a = min(seq) if abs(min(seq)) > abs(max(seq)) else max(seq)
        total = 0
        for x in seq:
            total += np.exp(x - a)
        return a + np.log(total)

    def likelihood(self, sequence):
        forward = self.forward(sequence)
        return self.log_sum_exp(forward[:,-1])

In [6]:
models = [HMM(), HMM()]
dataset = read_data()

# Training Model 
for i in range(len(CLASSES)):
    data = dataset[i]
    sum_ = 0
    for file in tqdm(data): # Decrease the size of training data to reduce time
        for seq in tqdm(file):
            models[i].update(seq)

## Part 2

Implement a basic two-class classifier using the HMMs constructed in the previous step.

In [7]:
def classify(model, seq):
    likelihood = np.array([model[i].likelihood(seq) for i in range(len(CLASSES))])
    return CLASSES[np.argmax(likelihood)]

tests = [dataset[0][-1][0], dataset[1][-1][0]]

for test in tests:
    print(classify(models, test))

xho
xho


## Q Experiment with different model orders (i.e., K = 3, 5, 7) and report your findings in terms of the classifier’s performance.

- The precision of classifier increases with the increase in the model order.
- Training time increases with the increase in the model order.
